In [ ]:
https://drive.google.com/file/d/1QrTOF1ToQ4plsSZbkBs3zOLkVt3MBlta/view?usp=sharing

In [2]:
#Start Code

In [31]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
import json
import cv2
import numpy as np

In [32]:

# Check if MPS is available
if torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")


print(device)

mps


#torch dataset


In [33]:


class KeypointsDataset(Dataset):
    # Initialization method
    def __init__(self, img_dir, data_file): 
        # Store the directory containing images
        self.img_dir = img_dir
        
        # Open the JSON data file and load its contents
        with open(data_file, "r") as f:
            self.data = json.load(f)  # Load JSON data into self.data

        # Define a sequence of transformations to be applied to each image
        self.transforms = transforms.Compose([
            transforms.ToPILImage(),  # Convert image to PIL format. PIL (Python Imaging Library) format standardizes image handling.
            transforms.Resize((224, 224)),  # Resize image to 224x224 pixels, a standard size for many pre-trained models.
            transforms.ToTensor(),  # Convert image to PyTorch tensor, scaling pixel values to the range [0, 1].
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize tensor with mean and std derived from ImageNet dataset.
        
        ])

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        item = self.data[idx]
        img = cv2.imread(f"{self.img_dir}/{item['id']}.png") #fetching img
        h,w = img.shape[:2]

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) #converting from b/w to color
        img = self.transforms(img)
        kps = np.array(item['kps']).flatten()
        kps = kps.astype(np.float32)

        kps[::2] *= 224.0 / w #adjusting x coord of the key points
        kps[1::2]*= 224.0 / h #adjusting y coord

        return img, kps

In [34]:
train_dataset = KeypointsDataset("data/images","data/data_train.json")
val_dataset = KeypointsDataset("data/images","data/data_val.json")

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=True)


Create Model


In [35]:
model = models.resnet50(pretrained= True)
model.fc = torch.nn.Linear(model.fc.in_features, 14*2) #we are using pretrained resnet model, to train on new data. fc means fully connected its the last later



In [36]:
model = model.to(device)

Train Model

In [37]:
#finding loss function we using mean square coz its regression
criterion= torch.nn.MSELoss()
optimizer= torch.optim.Adam(model.parameters(), lr=1e-4)

In [38]:
epochs=20
for epoch in range(epochs):
    for i,(img,kps) in enumerate(train_loader):
        img = img.to(device)
        kps = kps.to(device)

        optimizer.zero_grad()
        outputs = model(img)
        loss = criterion(outputs, kps)
        loss.backward()
        optimizer.step()

        if i%10 == 0:
            print(f"Epoch{epoch+1}/{epochs}, Step {i+1}/{len(train_loader)}, loss: {loss.item()}")


Epoch1/20, Step 1/829, loss: 14505.3623046875
Epoch1/20, Step 11/829, loss: 15205.7978515625
Epoch1/20, Step 21/829, loss: 14148.65234375
Epoch1/20, Step 31/829, loss: 13626.96484375
Epoch1/20, Step 41/829, loss: 14051.3330078125
Epoch1/20, Step 51/829, loss: 12286.5224609375
Epoch1/20, Step 61/829, loss: 12548.88671875
Epoch1/20, Step 71/829, loss: 12278.923828125
Epoch1/20, Step 81/829, loss: 12056.1201171875
Epoch1/20, Step 91/829, loss: 11366.4423828125
Epoch1/20, Step 101/829, loss: 11593.15234375
Epoch1/20, Step 111/829, loss: 10939.8427734375
Epoch1/20, Step 121/829, loss: 10010.6904296875
Epoch1/20, Step 131/829, loss: 9875.115234375
Epoch1/20, Step 141/829, loss: 10302.470703125
Epoch1/20, Step 151/829, loss: 8241.806640625
Epoch1/20, Step 161/829, loss: 8843.265625
Epoch1/20, Step 171/829, loss: 8441.1435546875
Epoch1/20, Step 181/829, loss: 8543.236328125
Epoch1/20, Step 191/829, loss: 7572.6240234375
Epoch1/20, Step 201/829, loss: 7321.81005859375
Epoch1/20, Step 211/829, l

In [40]:
torch.save(model.state_dict(), "keypoints_model.pth")